## 옮겨둔 이미지와 일치하는 json파일 가져오기

In [100]:
import os

folder_path = '/data/ephemeral/home/level2-cv-datacentric-cv-10/data/medical/OCR/img'  # 폴더 경로
files_ext = os.listdir(folder_path)  # 폴더 내의 파일 목록
# 확장자를 제외한 파일명만 추출
img_files = [os.path.splitext(file)[0] for file in files_ext]
len(img_files)

54

In [101]:
import os
import shutil

# json 파일이 있는 폴더 경로
json_source_folder = "../data/medical/OCR/01.라벨링데이터(Json)"

# 이동시킬 폴더
destination_json_folder = "../data/medical/OCR/json"

# json 파일을 복사하며 이미지 파일과 동일한 이름인 경우에만 복사
for root, dirs, files in os.walk(json_source_folder):
    for file in files:
        if file.endswith(".json"):
            source_file_path = os.path.join(root, file)
            destination_json_name = os.path.splitext(file)[0]  # 확장자 제외한 파일 이름만 가져오기
            destination_json_path = os.path.join(destination_json_folder, file)
            if destination_json_name in img_files:
                shutil.copy(source_file_path, destination_json_path)

모든 jpg 파일을 복사했습니다.


In [103]:
json_folder_path = '/data/ephemeral/home/level2-cv-datacentric-cv-10/data/medical/OCR/json'  # 폴더 경로
json_files_ext = os.listdir(json_folder_path)  # 폴더 내의 파일 목록
len(json_files_ext)

54

## json파일 합치기

In [12]:
info = {
    'year': 2024,
    'version': '1.0',
    'description': 'OCR Competition Data',
    'contributor': 'Naver Boostcamp',
    'url': 'https://aistages-api-public-prod.s3.amazonaws.com/app/Competitions/000273/data/data.tar.gz',
}
licenses = {
    'id': '1',
    'name': 'For Naver Boostcamp Competition',
    'url': None
}
categories = [{
    'id': 1,
    'name': 'word'
}]

In [19]:
import json
import os

# JSON 파일이 있는 폴더 경로
json_folder = '/data/ephemeral/home/level2-cv-datacentric-cv-10/data/medical/OCR/json/'
output_path = '/data/ephemeral/home/level2-cv-datacentric-cv-10/data/medical/OCR/json/new_json.json'

# 기존 정보
info = {
    'year': 2024,
    'version': '1.0',
    'description': 'OCR Competition Data',
    'contributor': 'Naver Boostcamp',
    'url': 'https://aistages-api-public-prod.s3.amazonaws.com/app/Competitions/000273/data/data.tar.gz',
}
licenses = {
    'id': '1',
    'name': 'For Naver Boostcamp Competition',
    'url': None
}
categories = [{'id': 1, 'name': 'word'}]

# COCO 데이터 초기화
img_id = 1
annotation_id = 1
images = []
annotations = []

for file_name in os.listdir(json_folder):
    if file_name.endswith('.json'):
        input_path = os.path.join(json_folder, file_name)

        with open(input_path, 'r') as f:
            file = json.load(f)
        image = {
            'id': img_id,
            'width': file['images'][0]['image.width'],
            'height': file['images'][0]['image.height'],
            'file_name': file['images'][0]["image.file.name"],
            "license": 1,
            "flickr_url": None,
            "coco_url": None,
            'data_captured': file['images'][0]["image.create.time"]
        }
        images.append(image)

        for ann_info in file['annotations']:
            min_x = ann_info["annotation.bbox"][0]
            min_y = ann_info["annotation.bbox"][1]
            width = ann_info["annotation.bbox"][2]
            height = ann_info["annotation.bbox"][3]

            coco_annotation = {
                "id": annotation_id,
                "image_id": img_id,
                "category_id": 1,
                "area": width * height,
                "bbox": [min_x, min_y, width, height],
                "iscrowd": 0
            }
            annotations.append(coco_annotation)
            annotation_id += 1

        img_id += 1

# 모든 데이터를 COCO 포맷으로 합치기
coco = {
    'info': info,
    'images': images,
    'annotations': annotations,
    'licenses': licenses,  # 리스트로 변환
    'categories': categories
}

# JSON 파일로 저장
with open(output_path, 'w') as f:
    json.dump(coco, f, indent=4)

# KeyError 0 나면, new_json.json가 이미 만들어져 있는 거임.

KeyError: 0

# coco -> ufo

In [20]:
from typing import Dict
import json
import datetime


now = datetime.datetime.now()
now = now.strftime('%Y-%m-%d %H:%M:%S')

input_path = '/data/ephemeral/home/level2-cv-datacentric-cv-10/data/medical/OCR/json/new_json.json'
output_path = '/data/ephemeral/home/level2-cv-datacentric-cv-10/data/medical/OCR/json/_new_json.json'

In [21]:
ufo = {
    'images': {}
}

In [26]:
def coco_bbox_to_ufo(bbox):
    min_x, min_y, width, height = bbox
    return [
        [min_x, min_y],
        [min_x + width, min_y],
        [min_x + width, min_y + height],
        [min_x, min_y + height]
    ]

def coco_to_ufo(file: Dict, output_path: str) -> None:
    anno_id = 1
    for annotation in file['annotations']:
        file_info = file['images'][int(annotation['image_id'])-1]
        image_name = file_info['file_name']
        if image_name not in ufo['images']:
            anno_id = 1
            ufo['images'][image_name] = {
                "paragraphs": {},
                "words": {},
                "chars": {},
                "img_w": file_info["width"],
                "img_h": file_info["height"],
                "tags": ["re-annotated"],
                "relations": {},
                "annotation_log": {
                    "worker": "",
                    "timestamp": now,
                    "tool_version": "LabelMe or CVAT",
                    "source": None
                    },
                "license_tag": {
                    "usability": True,
                    "public": False,
                    "commercial": True,
                    "type": None,
                    "holder": "Upstage"
                    }
                }
            
            # anno_id = 1
        ufo['images'][image_name]['words'][str(anno_id).zfill(4)] = {
            "transcription": "",
            "points":  coco_bbox_to_ufo(annotation["bbox"]),
            "orientation": "Horizontal",
            "language": None,
            "tags": None,
            "confidence": None,
            "illegibility": False
        }
        anno_id += 1

    with open(output_path, "w") as f:
        json.dump(ufo, f, indent=4)

In [27]:
with open(input_path, 'r') as f:
    file = json.load(f)
coco_to_ufo(file, output_path)

## 기존 json 파일과 합치기

In [28]:
# 기존 json
original_json_path = '/data/ephemeral/home/level2-cv-datacentric-cv-10/data/medical/ufo/train.json'
# custom_json파일
made_json_path = '/data/ephemeral/home/level2-cv-datacentric-cv-10/data/medical/OCR/json/_new_json.json'

In [50]:
import json
with open(original_json_path, 'r') as f:
    file_1 = json.load(f)
with open(made_json_path, 'r') as f:
    file_2 = json.load(f)

In [64]:
output_path ='/data/ephemeral/home/level2-cv-datacentric-cv-10/data/medical/ufo/_newest_.json'

combined_images = {}
combined_images.update(file_2)
combined_images.update(file_1)

combined_images['images']
with open(output_path, "w") as f:
    json.dump(combined_images, f, indent=4)